In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np
import seaborn as sns
import pyparetomixture.fit as pm
import pyparetomixture.type as pmt
from loguru import logger
import sys

logger.remove()
logger.add(sys.stdout, colorize=True, level="DEBUG")
None


In [8]:
# generate data
n = 10000
p = 0.95
alpha = 2.0
beta = 3.0  # beta > alpha


def generate_data(n, p, alpha, beta):
    x = np.random.binomial(1, p, n)
    rv_alpha = np.random.pareto(alpha, n)
    rv_beta = np.random.pareto(beta, n)
    return x * rv_alpha + (1 - x) * rv_beta


data = generate_data(n, p, alpha, beta)
logger.debug(
    f"Generated a sample of length {len(data)}. Mean: {np.mean(data)}, std: {np.std(data)}"
)


2023-04-05 20:28:24.035 | DEBUG    | __main__:<module>:16 - Generated a sample of length 10000. Mean: 0.9488068780987603, std: 2.2457484351925068


In [9]:
def pot(full_sample, k):
    full_sample.sort()
    return (full_sample[-k:] / full_sample[-k - 1]).copy()


def hill(sample):
    # hill estimate
    k = len(sample)
    assert (sample >= 1).all(), "not a pot sample"
    return k / (np.sum(np.log(sample)))


sample = pot(data, 200)
logger.debug(f"Generated pot sample. Min: {np.min(sample)}, max: {np.max(sample)}")


2023-04-05 20:28:24.101 | DEBUG    | __main__:<module>:14 - Generated pot sample. Min: 1.000728990410609, max: 12.468327486729253


In [20]:
def backtracking_line_search(
    sample, pmp, direction, p_greater_than_1, pre_normalize=True
):
    # backtracking line search
    # https://en.wikipedia.org/wiki/Backtracking_line_search

    old_ll = pm.loglikelihood(pmp, sample)
    logger.debug(
        f"Doing backtracking line search in direction {direction}. Old ll: {old_ll}"
    )

    if pre_normalize:
        direction = direction * (1 / np.linalg.norm(np.array([direction.dalpha, direction.dbeta, direction.dp])))
        logger.debug(f"We normalize our direction: {direction}")

    multiplier = 0.5
    n_iterations = 100
    for i in range(n_iterations):
        new_pmp = pmp + direction * multiplier

        logger.debug(f"New pmp in backtracking line search: {new_pmp}")
        # we ensure our new pmp do not surpass the thresholds we are given
        if not p_greater_than_1 and new_pmp.p >= 1:
            logger.debug(f"Encountered p >= 1 in {pmp} where p should be < 1")
            new_ll = -np.inf
        elif p_greater_than_1 and new_pmp.p <= 1:
            logger.debug(f"Encountered p <= 1 in {pmp} where p should be > 1")
            new_ll = -np.inf
        else:
            new_ll = pm.loglikelihood(new_pmp, sample)

        if new_ll == np.nan:
            logger.debug(f"Encountered np.nan ll with params {pmp}")
            new_ll = -np.inf

        logger.debug(f"Its likelihood is {new_ll}")

        if new_ll > old_ll:
            logger.debug(f"Found direction: {direction * multiplier}; new ll: {new_ll}")
            return direction * multiplier
        elif i > (n_iterations - 20) and new_ll > old_ll - 1e-5:
            logger.debug("Making a step for the sake of it.")
            return direction * multiplier

        multiplier *= 0.8
    raise Exception(
        f"backtracking line search did not converge in {n_iterations}, old_ll={old_ll}, new_ll={new_ll}, multiplier={multiplier}, direction={direction}"
    )


def search(sample, p_greater_than_1):
    if not p_greater_than_1:
        # p smaller than 1
        # alpha < hill estimate
        # D > 0
        # bias hill > 0
        # need a beta > alpha
        alpha = hill(sample)
        beta = alpha
        p = 1

        # theta = np.array([alpha, beta, p])
        # direction = np.array([1, 0, -1])
        pmp = pmt.ParetoMixtureParameters(alpha, beta, p)
        logger.debug(pmp)
        direction = pmt.DeltaParetoMixtureParameters(-1, 0, -1)

        direction = backtracking_line_search(sample, pmp, direction, p_greater_than_1, pre_normalize = False)
        pmp = pmp + direction 
        old_ll = pm.loglikelihood(pmp, sample)
        
        n_iterations = 100
        for _ in range(n_iterations):
            logger.debug("-" * 80)
            logger.debug(f"Got new pmp: {pmp}")
            direction = pm.gradient(pmp, sample)
            direction = pmt.DeltaParetoMixtureParameters(
                direction.dll_dalpha, direction.dll_dbeta, direction.dll_dp
            )
            direction = backtracking_line_search(
                sample, pmp, direction, p_greater_than_1, pre_normalize=False
            )
            pmp = pmp + direction
            new_ll = pm.loglikelihood(pmp, sample)
            if (new_ll - old_ll) < 1e-6:
                logger.debug(f"Got final pmp: {pmp}")
                return pmp
            else:
                logger.debug(f"old_ll={old_ll}, new_ll={new_ll}")
            old_ll = new_ll
        raise Exception(f"Did not finish in {n_iterations} iterations")


pmp = search(sample, False)
pmp

2023-04-05 21:47:15.982 | DEBUG    | __main__:search:65 - ParetoMixtureParameters(alpha=1.7427129151068517, beta=1.7427129151068517, p=1)
2023-04-05 21:47:15.983 | DEBUG    | __main__:backtracking_line_search:8 - Doing backtracking line search in direction DeltaParetoMixtureParameters(dalpha=-1, dbeta=0, dp=-1). Old ll: -314.7635954644528
2023-04-05 21:47:15.984 | DEBUG    | __main__:backtracking_line_search:21 - New pmp in backtracking line search: ParetoMixtureParameters(alpha=1.2427129151068517, beta=1.7427129151068517, p=0.5)
2023-04-05 21:47:15.984 | DEBUG    | __main__:backtracking_line_search:36 - Its likelihood is -320.9207414948673
2023-04-05 21:47:15.984 | DEBUG    | __main__:backtracking_line_search:21 - New pmp in backtracking line search: ParetoMixtureParameters(alpha=1.3427129151068518, beta=1.7427129151068517, p=0.6)
2023-04-05 21:47:15.985 | DEBUG    | __main__:backtracking_line_search:36 - Its likelihood is -317.2852276544995
2023-04-05 21:47:15.985 | DEBUG    | __main

/home/maurits/private/pareto_mixture/src/pyparetomixture/fit.py:92: RuntimeWarning: invalid value encountered in log
  dll_dalpha2 = np.sum(


ParetoMixtureParameters(alpha=1.5364571482715044, beta=1.6887430374296755, p=0.9954837901827764)

In [21]:
pm.gradient(pmp, sample)

Gradient(dll_dalpha=15.089892672686165, dll_dbeta=-0.007258314053641579, dll_dp=-5.409631303810023)